<a target="_blank" href="https://colab.research.google.com/github/jmanuelc87/nmp-autoavanza/blob/main/notebooks/MontePiedad_Rules.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

### Imports and Definitions

In [42]:
import os
import dotenv
import base64
import datetime
import itertools
import functools

from pydantic import BaseModel, Field

from operator import itemgetter

from rapidfuzz import fuzz, utils

from langchain_core.tools import tool
from langchain_core.output_parsers import PydanticToolsParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda, RunnablePassthrough, RunnableParallel, RunnablePick
from langchain_openai import ChatOpenAI
from langchain_community.document_loaders.image import UnstructuredImageLoader

In [43]:
env = dotenv.find_dotenv()
enable_open_ai = False

In [44]:
if enable_open_ai:
    os.environ["OPENAI_ORG_ID"] = dotenv.get_key(env, "org_id")
    os.environ["OPENAI_PROJECT_ID"] = dotenv.get_key(env, "proj_id")
    os.environ["OPENAI_API_KEY"] = dotenv.get_key(env, "api_key")
else:
    os.environ["OPENAI_ORG_ID"] = "***"
    os.environ["OPENAI_PROJECT_ID"] = "***"
    os.environ["OPENAI_API_KEY"] = "***"

os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGSMITH_API_KEY"] = dotenv.get_key(env, "langsmith_api_key")
os.environ["LANGSMITH_PROJECT"] = "pr-husky-stencil-39"

In [45]:
llm = ChatOpenAI(
    model="*",
    # model = "gpt-4.1-mini",
    temperature=0,
    base_url='http://localhost:1234/v1',
)

In [46]:
extract = [
    {
        "role": "system",
        "content": "Eres un asistente servicial, usando OCR extraes los attributos que se te indiquen de la imagen y verificas que sean correctos comparandolo con el documento {doc_name}.",
    },
    {
        "role": "user",
        "content": [
            {"type": "image_url", "image_url": {"url": "data:image/jpg;base64,{image}"}},
            {"type": "text", "text": "El documento {doc_name}: {page_content}"},
        ],
    },
]

In [47]:
def load_image(inputs):
    """Load image from file and encode it as base64."""
    image_path = inputs["filename"]
  
    def encode_image(image_path):
        with open(image_path, "rb") as image_file:
            return base64.b64encode(image_file.read()).decode('utf-8')
    image_base64 = encode_image(image_path)
    return image_base64

In [48]:
def load_unstructured_image(args):
    loader = UnstructuredImageLoader(args["filename"])
    data = loader.load()
    return data.pop().page_content

In [49]:
loader1 = RunnableLambda(load_unstructured_image)
loader2 = RunnableLambda(load_image)

### Extraccion Factura

In [50]:
file_url = "https://drive.google.com/file/d/1jumrzIoMQ4VJS2kCl_1E-_4bOOMlYmU6/view?usp=drive_link"
image_url = "https://drive.google.com/uc?export=download&id=1jumrzIoMQ4VJS2kCl_1E-_4bOOMlYmU6"
image1_path = "./data/bronze/BASE_AUTOAVANZA/documentos_clean/FAC_FRENTE/Caso 1_TK 62853-1 FAC_FRENTE_otsu.jpg"

In [51]:
prompt_extraction = ChatPromptTemplate(messages=extract)

In [52]:
class Factura(BaseModel):
    nombre_cliente: str = Field(description="El nombre del cliente")
    marca: str = Field(description="La marca")
    linea: str = Field(description="La linea")
    modelo: int = Field(description="El modelo")
    clase: str = Field(description="La clase")
    tipo: str = Field(description="El tipo")
    combustible: str = Field(description="El tipo de combustible")
    no_serie: str = Field(description="El numero de serie, NIV o VIN")
    no_motor: str = Field(description="El numero de motor")

In [53]:
llm_with_fac_structured_output = llm.with_structured_output(Factura)

In [54]:
extract_fac_chain = (
    {
        "page_content": loader1,
        "image": loader2,
        "doc_name": itemgetter("doc_name"),
    }
    | prompt_extraction
    | llm_with_fac_structured_output
)

In [55]:
# factura1 = extract_fac_chain.invoke(input={
#     "filename": image1_path,
#     "doc_name": "Factura",
# })

In [56]:
# print(factura1)

### Extraccion Tarjeta de Circulacion

In [57]:
file_url = "https://drive.google.com/file/d/1MeG8nX0r9eOkAK5xH0oN4z330WQDnYMk/view?usp=drive_link"
image_url = "https://drive.google.com/uc?export=download&id=1MeG8nX0r9eOkAK5xH0oN4z330WQDnYMk"
image2_path = "./data/bronze/BASE_AUTOAVANZA/documentos_clean/TC_FRENTE/Caso 1_TK 62853-4 TC_FRENTE_clean.jpg"

In [58]:
class TarjetaCirculacion(BaseModel):
    propietario: str = Field(description="El propietario o nombre del cliente")
    vehiculo: str = Field(description="El tipo de vehiculo")
    marca: str = Field(description="La marca")
    modelo: int = Field(description="El modelo del vehiculo")
    no_motor: str = Field(description="El numero de serie del motor")
    no_niv: str = Field(description="El numero de identificacion vehicular")
    expedicion: str = Field(description="La fecha de expedicion")
    vigencia: str = Field(description="La fecha de vigencia")
    placa: str = Field(description="El numero de placa")

In [59]:
llm_with_tc_structured_output = llm.with_structured_output(TarjetaCirculacion)

In [60]:
extract_tc_chain = (
    {
        "page_content": loader1,
        "image": loader2,
        "doc_name": itemgetter('doc_name'),
    }
    | prompt_extraction
    | llm_with_tc_structured_output
)

In [61]:
# tc1 = extract_tc_chain.invoke(input={
#     "filename": image2_path,
#     "doc_name": "Tarjeta de Circulacion",
# })

In [62]:
# print(tc1)

### Extraccion INE

In [63]:
file_url = "https://drive.google.com/file/d/1UKl8M0aMXJqN74iRPL9HFki8UvAdppa1/view?usp=drive_link"
image_url = "https://drive.google.com/uc?export=download&id=1UKl8M0aMXJqN74iRPL9HFki8UvAdppa1"
image3_path = "./data/bronze/BASE_AUTOAVANZA/Caso 1/TK 62853-2 INE_FRENTE.png"

In [64]:
class CredencialVotar(BaseModel):
    nombre: str = Field(description="El nombre de la persona")
    domicilio: str = Field(description="El domicilio de la persona")
    emision: int = Field(description="La fecha de emision")
    vigencia: int = Field(description="La fecha de vigencia")

In [65]:
llm_with_ine_structured_output = llm.with_structured_output(CredencialVotar)

In [66]:
extract_ine_chain = (
    {
        "page_content": loader1,
        "image": loader2,
        "doc_name": itemgetter('doc_name'),
    }
    | prompt_extraction
    | llm_with_ine_structured_output
)

In [67]:
# ine1 = extract_ine_chain.invoke(input={
#     "filename": image3_path,
#     "doc_name": "Credencial para Votar",
# })

In [68]:
# print(ine1)

# Reglas para validar documentos

Se definen ciertas reglas para validar documentos como se especifica a continuacion.

1. Se debe verificar la coincidencia del nombre del solicitante en la INE, Tarjeta de circulacion y Factura.

In [69]:
def compare_doc_names(factura: Factura, tarjeta_circulacion: TarjetaCirculacion, credencial_votar: CredencialVotar) -> str:
    """Compara el propietario o nombre en los documentos Factura, Tarjeta de Circulacion y Credencial para Votar

    Args:
        factura (Factura): La factura del vehiculo
        tarjeta_circulacion (TarjetaCirculacion): La Tarjeta de circulacion
        credencial_votar (CredencialVotar): La Credencial para Votar

    Returns:
        str: El resultado de la validacion es aprovado o rechazado 
    """
    wr1 = fuzz.WRatio(factura['nombre_cliente'], tarjeta_circulacion['propietario'], processor=utils.default_process)
    wr2 = fuzz.WRatio(factura['nombre_cliente'], credencial_votar['nombre'], processor=utils.default_process)
    wr3 = fuzz.WRatio(tarjeta_circulacion['propietario'], credencial_votar['nombre'], processor=utils.default_process)

    if wr1 >= 90. and wr2 >= 90. and wr3 >= 90.:
        return "El nombre del propietario es correcto!"
    elif 70. <= wr1 < 90. and 70. <= wr2 < 90. and 70. <= wr3 < 90.:
        return "Favor de verificar que el nombre aparezca correctamente en los tres documentos"
    else:
        return "El nombre no aparece correctamente en los documentos"

2. Se debe verificar el NIV es el mismo en la tarjeta de circulación y factura

In [70]:
def compare_vin(factura: Factura, tc:TarjetaCirculacion) -> str:
    """Compara el numero de serie, NIV o VIN en la factura y tarjeta de circulacion

    Args:
        factura (Factura): La Factura
        tc (TarjetaCirculacion): La Tarjeta de Circulacion

    Returns:
        str: El resultado de la validacion es aprovado o rechazado
    """

    wr = fuzz.WRatio(factura['no_serie'], tc['no_niv'])

    result = []

    if wr >= 90.:
        result.append("El NIV del vehiculo es correcto!")
    else:
        result.append("El NIV es distinto. Se tiene que rechazar el tramite!")

    wr2 = fuzz.WRatio(factura['no_motor'], tc['no_motor'])

    if wr >= 90.:
        result.append("El numero de serie del motor es correcto!")
    else:
        result.append("El numero de serie del motor no coincide")

    return result

3. Se debe comparar los datos del vehiculo (marca, linea y modelo) en la tarjeta de circulacion y factura

In [71]:
def compare_vehicle_data(factura: Factura, tc: TarjetaCirculacion) -> str:
    """Compara la marca, linea y modelo del vehiculo en la factura y tarjeta de circulacion

    Args:
        factura (Factura): La Factura
        tc (TarjetaCirculacion): La Tarjeta de Circulacion

    Returns:
        str: El resultado de la validacion es aprovado o rechazado
    """
    result = []

    wr = fuzz.WRatio(factura["marca"], tc["marca"])

    if wr < 90.0:
        label = f"La marca del vehiculo es distinto. Favor de verificar"
        result.append(label)

    wr = fuzz.WRatio(factura['linea'], tc['vehiculo'])

    if wr < 90.0:
        label = "La linea del vehiculo es distinta. Favor de verificar"
        result.append(label)

    wr = fuzz.WRatio(str(factura['modelo']), str(tc['modelo']))

    if wr < 90.0:
        label = "El modelo del vehiculo es distinto. Favor de verificar"
        result.append(label)

    if len(result) == 0:
        result.append("Los datos del vehiculo son correctos!")

    return result

9. Se debe verificar el numero de motor contra la tarjeta de circulacion

In [72]:
def compare_motor(factura: Factura, tc: TarjetaCirculacion) -> str:
    """Compara el motor en la Factura y Tarjeta de Circulacion

    Args:
        factura (Factura): La Factura
        tc (TarjetaCirculacion): La Tarjeta de Circulacion

    Returns:
        str: El resultado de la validacion es approvado o rechazado
    """
    wr = fuzz.WRatio(factura.no_motor, tc.no_motor)
    
    if wr < 90.0:
        return "El numero de motor del vehiculo es distinto. Favor de verificar"
    
    return "El numero de motor es correcto!"

In [73]:
def check_validity(ine: CredencialVotar) -> str:
    """Verfica la validez de la credencial para votar

    Args:
        ine (CredencialVotar): La credencial para votar

    Returns:
        str: El resultado de la validacion es approvador o rechazado
    """
    current_year = datetime.datetime.now().year
    if current_year > ine['vigencia']:
        return "La Credencial para Votar no esta vigente"
    return "La Credencial para Votar es vigente"

In [74]:
validate = [
    {
        "role": "system",
        "content": "Eres un asistente servicial.",
    },
    {
        "role": "user",
        "content": [
            {"type": "text", "text":"Realiza {comparation} en los documentos\n\n"},
            {"type": "text", "text":"La Factura: {factura}\n\n"},
            {"type": "text", "text":"La Tarjeta de Circulacion: {tarjeta_circulacion}\n\n"},
            {"type": "text", "text":"La Credencial para Votar: {credencial_votar}"},
        ],
    }
]

In [75]:
validate_prompt = ChatPromptTemplate.from_messages(validate)

In [76]:
tools=[compare_doc_names, compare_vin, compare_vehicle_data, check_validity]

In [77]:
llm_with_tools = llm.bind_tools(tools=tools)

In [ ]:
validations = [
    "la comparacion propietario o nombre de cliente",
    "la comparacion del numero de serie, VIN o NIV",
    "la comparacion de la marca, linea y modelo del vehiculo",
    "la verficacion de la validez de la credencial para votar"
]

In [79]:
def validate_documents(inputs, validations, tools):

    chain = validate_prompt | llm_with_tools | PydanticToolsParser(tools=tools)

    for comparation in validations:
        result = chain.invoke(
            input={
                "comparation": comparation,
                **inputs
            }
        )

        if isinstance(result[0], list):
            result = list(itertools.chain.from_iterable(result))

        yield result

In [80]:
validation = RunnableLambda(functools.partial(validate_documents, validations=validations, tools=tools))

In [81]:
chain = {
    "factura": {
        "filename": itemgetter("factura"),
        "doc_name": lambda x: "Factura",
    }
    | extract_fac_chain,
    "tarjeta_circulacion": {
        "filename": itemgetter("tc"),
        "doc_name": lambda x: "Tarjeta de Circulacion",
    }
    | extract_tc_chain,
    "credencial_votar": {
        "filename": itemgetter("ine"),
        "doc_name": lambda x: "Credencial para Votar"
    }
    | extract_ine_chain,
} | validation

In [82]:
for e in chain.stream({
    "factura": image1_path,
    "tc": image2_path,
    "ine": image3_path,
}):
    print(e)

['El nombre del propietario es correcto!']
['El NIV del vehiculo es correcto!', 'El numero de serie del motor es correcto!']
['Los datos del vehiculo son correctos!']
['El nombre del propietario es correcto!']
